In [42]:
import numpy as np
import tensorflow as tf
import scipy.stats as scipy
from scipy.stats import norm

from keras.models import Sequential
from keras.layers import Input, Dense, Concatenate, Dropout, Subtract, \
                        Flatten, Multiply, Lambda, Add, Dot
from keras.backend import constant
from keras import optimizers

#from keras.engine.topology import Layer
from keras.models import Model
from keras.layers import Input
from keras import initializers
from keras.constraints import max_norm
import keras.backend as K

import matplotlib.pyplot as plt

import copy


In [43]:
# this model is to solve the problem of portfolio hedging problem, in this code, we consider  quadratic hedging criterion as the risk measure

In [44]:
#Definition of neural networks for heding strategies
N=100 # time disrectization
m = 1 # dimension of price
d = 3 # number of layers in strategy
n = 32  # nodes in the first but last layers
strike = 100
sigma = 20 
T=1
S0 = 100
# this part just build the strategy layer for the whole structure 
layers = []
for j in range(N):
    for i in range(d):
        if i < d-1:    # before the output layer
            nodes = n
            layer = Dense(nodes, activation='tanh',trainable=True,       # considering relacing with ReLU 
                      kernel_initializer=initializers.RandomNormal(0,1),#kernel_initializer='random_normal',
                      bias_initializer='random_normal',
                      name=str(i)+str(j))
        else:
            nodes = m    # last layer
            layer = Dense(nodes, activation='linear', trainable=True,
                          kernel_initializer=initializers.RandomNormal(0,0.1),#kernel_initializer='random_normal',
                          bias_initializer='random_normal',
                          name=str(i)+str(j))
        layers = layers + [layer]


In [45]:
# for simplify I just copy the bs model code so it seems messy
def BS(S0, strike, T, sigma):
    return S0*scipy.norm.cdf((np.log(S0/strike)+0.5*T*sigma**2)/(np.sqrt(T)*sigma))-strike*scipy.norm.cdf((np.log(S0/strike)-0.5*T*sigma**2)/(np.sqrt(T)*sigma))

priceBS=BS(S0,strike,T,sigma)


In [46]:
#Implementing the loss function

#the initial hedging being 0, and the increments of the log price process 
price = Input(shape=(m,))
hedge = Input(shape=(m,))
hedgeeval = Input(shape=(m,))
premium = Input(shape=(m,))   # input parameter

inputs = [price]+[hedge]+[hedgeeval]+[premium]
outputhelper=[]

premium = Dense(m, activation='linear', trainable=True,
                kernel_initializer=initializers.RandomNormal(0,1),#kernel_initializer='random_normal',
                bias_initializer=initializers.RandomNormal(0,1))(premium)

for j in range(N):
    strategy = price
    strategyeval=hedgeeval 
    for k in range(d):
        strategy= layers[k+(j)*d](strategy) # add the hedging strategy at j 
        strategyeval=layers[k+(j)*d](strategyeval) 
    incr = Input(shape=(m,))   #increment of the log price
    logprice= Lambda(lambda x : K.log(x))(price) 
    logprice = Add()([logprice, incr])
    pricenew=Lambda(lambda x : K.exp(x))(logprice)# creating the price at time j+1
    priceincr=Subtract()([pricenew, price]) 
    hedgenew = Multiply()([strategy, priceincr])
    hedge = Add()([hedge,hedgenew]) # building up the discretized stochastic integral
    inputs = inputs + [incr]
    outputhelper = outputhelper + [strategyeval]
    price=pricenew
payoff= Lambda(lambda x : 0.5*(K.abs(x-strike)+x-strike))(price) 
outputs = Subtract()([payoff,hedge]) 
outputs = Subtract()([outputs,premium]) # payoff minus price minus hedge 
outputs= [outputs] + outputhelper +[premium]  
outputs = Concatenate()(outputs)            # quadratic hedging criterion

model_hedge_strat = Model(inputs=inputs, outputs=outputs)

In [47]:
from keras import losses
def custom_loss(y_true,y_pred):
    #return losses.mean_squared_error(y_true[0], y_pred[0])
    z = y_pred[:,0]-y_true[:,0]
    z=K.mean(K.square(z))
    return z



In [48]:
gamma = 1.0
grid = [(i/N)**gamma*T for i in range(N+1)]

Ktrain = 10**5
initialprice = S0

# xtrain consists of the price S0, 
#the initial hedging being 0, and the increments of the log price process 
xtrain = ([initialprice*np.ones((Ktrain,m))] +
          [np.zeros((Ktrain,m))]+
          [np.ones((Ktrain,m))] +
          [priceBS*np.ones((Ktrain,m))]+
          [np.random.normal(-(sigma)**2/2*(grid[i+1]-grid[i]),sigma*np.sqrt(grid[i+1]-grid[i]),(Ktrain,m)) for i in range(N)])

ytrain=np.zeros((Ktrain,1+N))


In [49]:
xtrain

[array([[100.],
        [100.],
        [100.],
        ...,
        [100.],
        [100.],
        [100.]]),
 array([[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]]),
 array([[1.],
        [1.],
        [1.],
        ...,
        [1.],
        [1.],
        [1.]]),
 array([[100.],
        [100.],
        [100.],
        ...,
        [100.],
        [100.],
        [100.]]),
 array([[-1.88144202],
        [-3.03285502],
        [-1.39356882],
        ...,
        [-4.22778563],
        [-2.94670002],
        [-2.09535535]]),
 array([[ 1.22823399],
        [-4.44058881],
        [-1.72875555],
        ...,
        [-2.69888963],
        [-3.98599762],
        [ 3.4726641 ]]),
 array([[-0.49308908],
        [-0.74642629],
        [-4.99559407],
        ...,
        [-2.03055339],
        [-2.58975112],
        [-3.77736262]]),
 array([[-1.88273154],
        [-1.68221527],
        [-0.47265141],
        ...,
        [ 0.99638496],
        [ 0.1244

In [50]:
logincrements = xtrain[4:4+N]
hedge = np.zeros(Ktrain)
price = S0*np.ones((Ktrain,N))
for k in range(N-1):
    helper = logincrements[k][:,]
    helper = helper.transpose()
    price[:,k+1] = price[:,k]*np.exp(helper[:])
    hedge[:] = hedge[:] + scipy.norm.cdf((np.log(price[:,k]/strike)+0.5*(T-grid[k+1])*sigma**2)/(np.sqrt(T-grid[k+1])*sigma))*(price[:,k+1]-price[:,k])
hedge[:]= hedge[:]-0.5*(np.abs(price[:,N-1]-strike)+(price[:,N-1]-strike))+ priceBS
plt.show()
print(np.std(hedge))
print(np.mean(hedge))


4.0894510729670906e-13
3.8113512346171774e-16
